# Reference model constraint

In [29]:
# how many n soundings
# create matix with n columns
# use standard/input parameters
# allow user to input own parameters at certain location

## cell by sounding n or by coordinates??

In [260]:
#import libraries
import pandas as pd
import numpy as np
import xarray as xr

In [261]:
## User requirement: enter the folder location where the .dat file is saved
folder= str(r"C:/Users/u67397/AnacondaProjects/aem/input_data/")

## User requirement: enter the name of the second .dat file (without the ".dat" text)
dat_name= str("AA140005_Line1080")

## User requirement: enter the reference model conductivity specified for result file
reference_EC1= 0.001

## User requirement: enter the number of layers result file
layers= 30

## User requirement: enter the name of the second .dat file (without the ".dat" text)
con_name= str("galeisbstdem")

In [262]:
## User requirement: enter the folder location where to save files into

output_folder= str(r"C:/Users/u67397/AnacondaProjects/aem/output_data")

In [263]:
#Enter additional information. Create tests. e.g. 3 layers at xxx location with xxx EC values

In [264]:
dat= pd.read_fwf(folder+dat_name+".dat", header=None)
datlen= len(dat.index)

In [265]:
#thickness from con file
con = pd.read_csv (folder+con_name+".con", header=None) #import control file
con.replace(regex=True,to_replace=r'\t', value=r'', inplace=True)#remove tabs
tlayers=con.loc[con[0].str.contains('Thickness      = ')] #create a dataframe that contains layer thickness saved in confile (.con)
thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')
thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan)
thicknessoflayers.dropna(axis=1,inplace=True)
thicknessoflayers=thicknessoflayers.transpose().astype(float)
thicknessoflayers=np.cumsum(thicknessoflayers)
thicknessoflayers=thicknessoflayers.transpose()
thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))
thicknessoflayers=pd.DataFrame(thicknessoflayers)

In [266]:
EC=pd.Series(reference_EC1)
EC=(pd.np.tile(EC, (datlen, layers)))
EC=pd.DataFrame(EC)

In [267]:
coords_m=dat.iloc[:,1:3]
EC_m= xr.Dataset(EC.iloc[0:9,0:9])
thickness_m=xr.Dataset(thicknessoflayers.iloc[0:9,0:9])
dat_m=xr.Dataset(dat.iloc[0:9,0:9])


In [268]:
#join datafile and layer thickness dataframes
alldata=pd.concat([dat,thicknessoflayers,EC], axis=1, join='outer',ignore_index=True)
##Weighting - adjust to hit reference model. set out to side and botom x layers. 
## input constraint to relax in horiz space


In [269]:
alldata=alldata.set_index([2,3], drop=False)
alldata['all_sumcoord']=alldata[2]+alldata[3]

In [270]:
#open constraint csv
constraint_name=str("input_test") 
constraint_data= pd.read_csv(folder+constraint_name+".csv") #import control file
constraint_data['const_sumcoord']=constraint_data['x']+constraint_data['y']
constraint_data=constraint_data.set_index(['ID'], drop=False)

In [271]:
constraint_data

,ID,x,y,zt,zb,v,const_sumcoord
ID,,,,,,,
erin,erin,220955.6,6867443.6,0,-10,0.05,7088399.2
rules,rules,220956.6,6867444.6,-10,-20,0.10,7088401.2


In [274]:
matched=pd.DataFrame([0,0,0]).transpose().set_index([0])

for x in range(len(constraint_data)):
    g=(constraint_data.ID[x])
    h=(constraint_data.const_sumcoord[x])
    alldata[str(g)]= np.abs(alldata.all_sumcoord-h) 
    matched1=np.argmin(alldata[str(g)])
    e_matched=matched1[0]
    n_matched=matched1[1]
    matched2=pd.DataFrame([g,float(e_matched),float(n_matched)]).transpose().set_index([0]).astype(float)
    matched=pd.concat([matched,matched2], axis=0)

matched=matched.rename(columns={1:'E_matched',2:'N_matched'}).drop([0])
print(matched)

NameError: name 'prin' is not defined

In [43]:
constraint_data2= pd.concat([constraint_data, matched],axis=1)
constraint_data2
constraint_data2=constraint_data2.set_index(['E_matched','N_matched'])
constraint_data2

,,ID,x,y,zt,zb,v,const_sumcoord
E_matched,N_matched,,,,,,,
220955.6,6867443.6,erin,220955.6,6867443.6,0,-10,0.05,7088399.2
220958.6,6867443.7,rules,220956.6,6867444.6,-10,-20,0.10,7088401.2


In [124]:
#create subset of three dataframes
EC_m=EC.iloc[0:5,0:5]
thickness_m=thicknessoflayers.iloc[0:5,0:5]
coords_m=pd.DataFrame(dat.iloc[0:5,2:3])

#create array 
testarray=np.array([[thickness_m],[EC_m]])

In [49]:
import xarray

In [223]:
coords_test= coords_m.transpose()
#coords_test=(coords_test.values.tolist())
coords_test=np.array(coords_test).squeeze()

[ 220926.1  220929.   220931.9  220934.9  220937.8]


In [231]:
# thickness_test=(thickness_m[0:1]).values.tolist()
# #thickness_test=tuple(thickness_test)
# thickness_test

thickness_test= thickness_m[0:1]
#coords_test=(coords_test.values.tolist())
thickness_test=np.array(thickness_test).squeeze()
thickness_test

array([  3.  ,   6.3 ,   9.92,  13.92,  18.32])

In [233]:
coords={'location': coords_test, 'depth':thickness_test}
ec_da = xarray.DataArray(EC_m, dims=('location', 'depth'), coords=coords)
ec_da

<xarray.DataArray (location: 5, depth: 5)>
array([[ 0.001,  0.001,  0.001,  0.001,  0.001],
       [ 0.001,  0.001,  0.001,  0.001,  0.001],
       [ 0.001,  0.001,  0.001,  0.001,  0.001],
       [ 0.001,  0.001,  0.001,  0.001,  0.001],
       [ 0.001,  0.001,  0.001,  0.001,  0.001]])
Coordinates:
  * location  (location) float64 2.209e+05 2.209e+05 2.209e+05 2.209e+05 ...
  * depth     (depth) float64 3.0 6.3 9.92 13.92 18.32

In [278]:
coords={'location': coords_test, 'layer': [1, 2, 3, 4, 5]}
thickness_da = xarray.DataArray(thickness_m, dims=('location', 'layer'), coords=coords)

ValueError: conflicting sizes for dimension 'location': length 10 on the data but length 5 on coordinate 'location'

In [279]:
dataset = xarray.Dataset({'ec': ec_da, 'thickness_name': thickness_da})
dataset

<xarray.Dataset>
Dimensions:         (depth: 5, layer: 5, location: 5)
Coordinates:
  * location        (location) float64 2.209e+05 2.209e+05 2.209e+05 ...
  * depth           (depth) float64 3.0 6.3 9.92 13.92 18.32
  * layer           (layer) int32 0 1 2 3 4
Data variables:
    ec              (location, depth) float64 0.001 0.001 0.001 0.001 0.001 ...
    thickness_name  (location, layer) float64 3.0 6.3 9.92 13.92 18.32 3.0 ...

In [291]:
def test ():
    if depth < 10:
        ec == 0.05
    else:
        ec == 0.01

dataset.groupby('depth').apply(test())

NameError: name 'depth' is not defined

In [259]:
test= dataset.sel(location=220926.1).ec.where(thickness_da <9)
# test = 0.05
test

<xarray.DataArray (depth: 5, location: 5, layer: 5)>
array([[[ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan]],

       [[ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan]],

       [[ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan]],

       [[ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan,    nan],
        [ 0.001,  0.001,    nan,    nan, 

In [207]:
dataset.sel(location=220926.1).thickness_name <10

<xarray.DataArray 'thickness_name' (thickness: 5)>
array([ True,  True,  True, False, False], dtype=bool)
Coordinates:
    location   float64 2.209e+05
  * thickness  (thickness) int32 4 5 6 7 8

In [88]:
dataset.ec.isel_points(location=(dataset.thickness_name < 10).location)

IndexError: index 51 is out of bounds for axis 0 with size 5

In [78]:
dataset.where(dataset.sel(location=51).thickness_name < 10).fillna(100)

<xarray.Dataset>
Dimensions:         (depth: 5, location: 5, thickness: 5)
Coordinates:
  * location        (location) int32 51 52 53 54 55
  * depth           (depth) int32 2 3 4 5 6
  * thickness       (thickness) int32 4 5 6 7 8
Data variables:
    ec              (location, depth, thickness) float64 0.001 0.001 0.001 ...
    thickness_name  (location, thickness) float64 3.0 6.3 9.92 100.0 100.0 ...

In [65]:
dataset.location[:] = [5, 6, 7, 8, 9]

TypeError: IndexVariable values cannot be modified

In [17]:
#create and save a .csv that contains depth of investigation data for input into Discover PA software
alldata.to_csv(output_folder+'/depth_data.csv') #save doi data as csv

In [ ]:

#coords_m=pd.DataFrame(dat.iloc[0:5,2:4])

# test=pd.concat([coords_m,thickness_m,EC_m],axis=1,ignore_index=True).set_index([0,1])
# test=pd.concat([test,constraint_data2],axis=1)
# test 


# thickness_m=pd.concat([coords_m,thickness_m],axis=1,ignore_index=True)#.set_index([0,1])
# array=np.array([[coords_m],[thickness_m],[EC_m]])
# array.shape

In [ ]:
# for row in constraint_data:
#     print(row)
    #sample[str(row)]= 0

#print(sample)
    
    
#     print('loop1_start')
#     print(row)
#     print('loop1_ends')
#     for blah in constraint_data['const_sumcoord']:
#         print('loop2_starts')
#         print (blah)
#         print('loop2_ends')
# #         while x>0:
#         for rah in sample:
#             sample[str(row)]=np.abs(test.all_sumcoord-blah)
# #                 x=x-1
#             print(sample)
                

In [ ]:
## find coordinate
## enter layer thickness (e.g 3)
## extrapolate between the points to maximum layers (e.g. 30)

In [136]:
# array=np.array([[dat],[thicknessoflayers],[EC]],ndmin=3)
# array.shape
# #test=np.ndarray(array)
# array[[0,0,0],[[0,0,0],[0,0,0]]]

In [ ]:
# #create a dataframe that contains layer thickness saved in confile (.con)
# tlayers=con.loc[con[0].str.contains('Thickness      = ')]
# thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')
# thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan)
# thicknessoflayers.dropna(axis=1,inplace=True)
# thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))
# #thicknessoflayers=pd.DataFrame(thicknessoflayers)
# thicknessoflayers

In [ ]:
# thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))


In [ ]:
def coord(easting,northing):
    easting=int(easting)
    northing=int(northing)
    print(type(easting))

coord('1','2')

In [ ]:
def print_text():
    print("complete")

In [ ]:
a = [[1, 2, 3, 4], [5, 6], [7, 8, 9]]
for i in range(len(a)):
    for j in range(len(a[i])):
        print(a[i][j], end=' ')
    print()

In [119]:
# EC_x=xr.Dataset([EC_m],[thickness_m])
# EC_x

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().